In [1]:
import cv2, pywt
import numpy as np

# read image (grayscale for demo). For RGB, split channels and do this per channel.
img = cv2.imread("images/lg-cat.webp", cv2.IMREAD_GRAYSCALE).astype(np.float32)

# 1-level 2D DWT with Haar (you can use 'db2', 'bior4.4', etc.)
LL, (LH, HL, HH) = pywt.dwt2(img, wavelet='haar')

print(LL.shape, LH.shape, HL.shape, HH.shape)  # each ~ (H//2, W//2)

# reconstruct
recon = pywt.idwt2((LL, (LH, HL, HH)), wavelet='haar')


(988, 1500) (988, 1500) (988, 1500) (988, 1500)


In [2]:
print(LL)

[[28.      28.      28.      ... 60.      60.      60.     ]
 [28.      28.      28.      ... 60.      60.      60.     ]
 [28.      28.      28.      ... 60.      60.      60.     ]
 ...
 [85.      85.      85.      ... 80.      80.      80.     ]
 [83.      83.      83.      ... 80.      80.      80.     ]
 [81.99999 81.99999 81.99999 ... 80.      80.      80.     ]]


In [3]:
# level-2 decomposition
coeffs = pywt.wavedec2(img, wavelet='haar', level=2)
# coeffs[0] = LL at level 2
# coeffs[1] = (LH2, HL2, HH2), coeffs[2] = (LH1, HL1, HH1)

In [4]:
LL2, (LH2, HL2, HH2) = coeffs[0], coeffs[1]
LL2

array([[ 56.      ,  56.      ,  56.      , ..., 122.      , 120.      ,
        120.      ],
       [ 56.      ,  56.      ,  56.      , ..., 123.      , 120.      ,
        120.      ],
       [ 56.      ,  56.      ,  56.      , ..., 126.999985, 123.99999 ,
        123.99999 ],
       ...,
       [179.99998 , 179.99998 , 179.      , ..., 153.99998 , 150.      ,
        150.      ],
       [172.99998 , 172.99998 , 172.99998 , ..., 161.99998 , 156.      ,
        156.      ],
       [164.99998 , 164.99998 , 164.99998 , ..., 163.99998 , 160.      ,
        160.      ]], shape=(494, 750), dtype=float32)

In [5]:
import cv2
import numpy as np
from typing import Union, Tuple

def to_binary(
    img: Union[str, np.ndarray],
    size: Union[None, int, Tuple[int, int]] = None,   # int => square, tuple => (width, height)
    method: str = "otsu",                             # "otsu" or "fixed"
    threshold: int = None,                            # used when method="fixed"
    invert: bool = False                              # True => white background, black foreground
) -> np.ndarray:
    """
    Convert image to binary (0/255) with optional resize.

    Returns: uint8 array of shape (H, W) with values {0, 255}.
    """
    # 1) Load (if path) and ensure grayscale
    if isinstance(img, str):
        src = cv2.imread(img, cv2.IMREAD_UNCHANGED)
        if src is None:
            raise ValueError(f"Could not read image: {img}")
    else:
        src = img.copy()

    if src.ndim == 3 and src.shape[2] == 4:       # drop alpha if present
        src = cv2.cvtColor(src, cv2.COLOR_BGRA2BGR)
    if src.ndim == 3:                             # assume BGR; OK for most OpenCV images
        gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    else:
        gray = src

    # 2) Resize if requested (OpenCV uses (width, height))
    if size is not None:
        if isinstance(size, int):
            size = (size, size)
        w, h = size
        interp = cv2.INTER_AREA if (gray.shape[1] > w or gray.shape[0] > h) else cv2.INTER_LINEAR
        gray = cv2.resize(gray, (w, h), interpolation=interp)

    # 3) Make sure it's 8-bit for thresholding
    if gray.dtype != np.uint8:
        gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # 4) Threshold
    base_flag = cv2.THRESH_BINARY_INV if invert else cv2.THRESH_BINARY
    if method.lower() == "otsu":
        _, binary = cv2.threshold(gray, 0, 255, base_flag | cv2.THRESH_OTSU)
    elif method.lower() == "fixed":
        if threshold is None:
            raise ValueError("Provide 'threshold' (0–255) when method='fixed'.")
        thr = int(np.clip(threshold, 0, 255))
        _, binary = cv2.threshold(gray, thr, 255, base_flag)
    else:
        raise ValueError("method must be 'otsu' or 'fixed'.")

    return binary

In [6]:
bw1 = to_binary("images/f1-logo.png", size=(256, 256), method="otsu")
bw1

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], shape=(256, 256), dtype=uint8)

In [7]:
bw1 = bw1/255

bw1

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], shape=(256, 256))

In [8]:
cv2.imshow("Binary Image", bw1)
cv2.waitKey(0)

-1

In [9]:
bw2 = to_binary("images/f1-logo.png", size=128, method="fixed", threshold=127, invert=True)

cv2.imshow("Binary Image", bw2)
cv2.waitKey(0)

-1

In [10]:
import numpy as np

def image_to_bitstream(
    img,
    size=None,
    method="otsu",
    threshold=None,
    invert=False,
    pack=False,           # True -> return packed bytes
    bitorder="big"        # 'big' or 'little' for packbits
):
    """
    Returns (bits, shape). If pack=True, 'bits' is uint8 bytes; else uint8 {0,1}.
    'shape' is (H, W) of the binarized image for reconstruction later.
    """
    bw = to_binary(img, size=size, method=method, threshold=threshold, invert=invert)
    bits = (bw.ravel() // 255).astype(np.uint8)   # 255->1, 0->0

    if pack:
        return np.packbits(bits, bitorder=bitorder), bw.shape
    else:
        return bits, bw.shape

def bitstream_to_image(
    bits,
    shape,
    unpack=False,
    bitorder="big"
):
    """
    Reconstruct a binary image from a bitstream and target shape.
    If unpack=True, 'bits' is a bytes-like array packed by np.packbits.
    """
    if unpack:
        n = shape[0] * shape[1]
        bits = np.unpackbits(bits, bitorder=bitorder)[:n]
    img = (bits.reshape(shape).astype(np.uint8) * 255)
    return img

In [11]:
bits, shape = image_to_bitstream("images/f1-logo.png", size=64, method="otsu", pack=False)

In [12]:
bits

array([1, 1, 1, ..., 1, 1, 1], shape=(4096,), dtype=uint8)

In [13]:
img = bitstream_to_image(bits, shape, unpack=False)

In [14]:
cv2.imshow("Binary Image", img)
cv2.waitKey(0)

-1

## Bitstream Encryption

In [15]:
# pip install cryptography
import os, struct, numpy as np
from typing import Literal
from cryptography.hazmat.primitives.ciphers.aead import AESGCM, ChaCha20Poly1305
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt

# ---- Parameters ----
SCHEME_INFO = b"wm:bits-aead-scrypt:v1"    # domain separation (AAD)
SALT_LEN   = 16
NONCE_LEN  = 12
KEY_LEN    = 32
# Scrypt work factors (tune for your environment)
SCRYPT_N   = 2**14
SCRYPT_R   = 8
SCRYPT_P   = 1

def _kdf_scrypt(password: bytes, salt: bytes) -> bytes:
    kdf = Scrypt(salt=salt, length=KEY_LEN, n=SCRYPT_N, r=SCRYPT_R, p=SCRYPT_P)
    return kdf.derive(password)

def _get_aead(alg: Literal["aesgcm","chacha20poly1305"], key: bytes):
    if alg == "aesgcm":
        return AESGCM(key), 1  # algo id 1
    elif alg == "chacha20poly1305":
        return ChaCha20Poly1305(key), 2  # algo id 2
    else:
        raise ValueError("alg must be 'aesgcm' or 'chacha20poly1305'.")

def encrypt_bitstream_with_password(
    bits: np.ndarray,
    password: str | bytes,
    alg: Literal["aesgcm","chacha20poly1305"] = "chacha20poly1305",
    bitorder: Literal["big","little"] = "big"
) -> bytes:
    """
    Encrypt a 0/1 uint8 bitstream with a password.
    Returns a bytes 'package' containing header + salt + nonce + ciphertext(tag).
    """
    if bits.dtype != np.uint8:
        bits = bits.astype(np.uint8)
    if not np.array_equal(bits, bits & 1):
        raise ValueError("bits must be 0/1 uint8 array")

    # Pack bits -> bytes
    bit_len = bits.size
    pt = np.packbits(bits, bitorder=bitorder).tobytes()

    # KDF
    salt  = os.urandom(SALT_LEN)
    key   = _kdf_scrypt(password if isinstance(password, bytes) else password.encode(), salt)

    # AEAD
    aead, alg_id = _get_aead(alg, key)
    nonce = os.urandom(NONCE_LEN)
    aad = SCHEME_INFO  # bind ciphertext to this context
    ct = aead.encrypt(nonce, pt, aad)

    # Package format:
    # version(1) | alg_id(1) | bit_len(4, big-endian) | salt(16) | nonce(12) | ct(var)
    header = b"\x01" + bytes([alg_id]) + struct.pack(">I", bit_len) + salt + nonce
    return header + ct

def decrypt_bitstream_with_password(
    package: bytes,
    password: str | bytes,
    bitorder: Literal["big","little"] = "big"
) -> np.ndarray:
    """
    Decrypt a package produced by encrypt_bitstream_with_password and return 0/1 bits.
    """
    # Parse header
    if len(package) < 1 + 1 + 4 + SALT_LEN + NONCE_LEN:
        raise ValueError("package too short")
    off = 0
    version = package[off]; off += 1
    if version != 0x01:
        raise ValueError("unsupported version")
    alg_id = package[off]; off += 1
    bit_len = struct.unpack(">I", package[off:off+4])[0]; off += 4
    salt = package[off:off+SALT_LEN]; off += SALT_LEN
    nonce = package[off:off+NONCE_LEN]; off += NONCE_LEN
    ct = package[off:]
    if not ct:
        raise ValueError("missing ciphertext")

    # Rebuild AEAD
    key = _kdf_scrypt(password if isinstance(password, bytes) else password.encode(), salt)
    alg = "aesgcm" if alg_id == 1 else "chacha20poly1305" if alg_id == 2 else None
    if alg is None: raise ValueError("unknown alg_id")
    aead, _ = _get_aead(alg, key)

    pt = aead.decrypt(nonce, ct, SCHEME_INFO)

    # Unpack bytes -> bits and trim to exact bit_len
    unpacked = np.unpackbits(np.frombuffer(pt, dtype=np.uint8), bitorder=bitorder)
    return unpacked[:bit_len].astype(np.uint8)


In [20]:
# From earlier:
# bits, shape = image_to_bitstream("logo.png", size=64, method="otsu", pack=False)

password = "My$trongPa55"
package = encrypt_bitstream_with_password(bits, password, alg="chacha20poly1305")
# If your embedding layer requires bits, convert package bytes -> bits:
pkg_bits = np.unpackbits(np.frombuffer(package, dtype=np.uint8), bitorder="big")

# ...embed pkg_bits into your watermark...
# On extraction, recover pkg_bits (in original order), then:
package_bytes = np.packbits(pkg_bits, bitorder="big").tobytes()
rec_bits = decrypt_bitstream_with_password(package_bytes, "My$trongPa55")

# rec_bits == bits (0/1), ready for bitstream_to_image(rec_bits, shape)


In [21]:
pkg_bits.shape, bits.shape, rec_bits.shape

((4496,), (4096,), (4096,))

In [18]:
# For your large encrypted package, convert to bytes instead
def binary_to_bytes(bits, bitorder="big"):
    """Convert binary array to bytes"""
    return np.packbits(bits, bitorder=bitorder)

def bytes_to_hex(byte_array):
    """Convert bytes to hex string for display"""
    return byte_array.tobytes().hex()

# Convert your large binary to bytes
pkg_bytes = binary_to_bytes(pkg_bits)
print(f"Package as bytes: {pkg_bytes}")
print(f"Package as hex: {bytes_to_hex(pkg_bytes)}")
print(f"Size: {len(pkg_bytes)} bytes")

Package as bytes: [  1   2   0   0  16   0  18 237 204 151  10 115 154 253  35 209 192  52
 205 102 206 238 123 195  44 111  11  30 239  81  36 239  38 220  54   0
 205 172  46  95 111 235  49   7  75 230 105  43 228  84  46 205  39 183
 102 201   9  13 183 135 178  12  68  21 188  27 246 164 218  24 192  47
  44 248  47 255  65 140 115  24  28  15  37  16 152 133 126 129 230  55
  66 219 127  76 138  35 117  92 122   0  16 232  22  34 251 202  81  17
 132 140  85 238 117   2 232  83  32  12 104 237  28 215 144 182 178 174
 223 234   5  46  69  73 167 213 175  36 172 118 161  97  77 236 139   4
 188 114  73  80 180  55  18  76 241 122 170 212  91  38 119  59  74  77
 218 146  49 161 109  94 219 129 133 106 222 210 252 140 113  27 195 169
 101   4 215  61 230 142 227 224 153  39 109 212 254 205  28 164 191   6
 247 107 171 176  79 190 174  39 128 238 245 104  41  51  25 255  28 152
  80  33 223  56 185 174  53 226 185   0 108  44 123 231 182 240  56 182
  91  62  44 128  72  82  76 189 

encryption module

now need to check if it my watermarking need this encryption

In [8]:
import numpy as np
from PIL import Image
import pywt
import cv2

# ---------------------------
# Utility: PSNR (optional)
# ---------------------------
def psnr(img_a: np.ndarray, img_b: np.ndarray) -> float:
    a = img_a.astype(np.float64)
    b = img_b.astype(np.float64)
    mse = np.mean((a - b) ** 2)
    if mse <= 1e-12:
        return 100.0
    return 20.0 * np.log10(255.0 / np.sqrt(mse))

# ---------------------------
# YCbCr I/O helpers
# ---------------------------
def rgb_to_ycbcr_arrays(pil_img: Image.Image):
    im = pil_img.convert("RGB")
    ycbcr = im.convert("YCbCr")
    Y  = np.array(ycbcr.getchannel(0), dtype=np.float64)
    Cb = np.array(ycbcr.getchannel(1), dtype=np.uint8)
    Cr = np.array(ycbcr.getchannel(2), dtype=np.uint8)
    return Y, Cb, Cr

def merge_ycbcr_to_rgb(Y: np.ndarray, Cb: np.ndarray, Cr: np.ndarray) -> Image.Image:
    Y8 = np.rint(np.clip(Y, 0, 255)).astype(np.uint8)
    ycbcr = Image.merge(
        "YCbCr",
        (Image.fromarray(Y8), Image.fromarray(Cb), Image.fromarray(Cr))
    )
    return ycbcr.convert("RGB")

# ---------------------------
# Watermark (32x32) -> bits
# ---------------------------
def wm32_to_bits(wm_img: Image.Image, threshold: int = 127) -> np.ndarray:
    """
    Convert a mono 32x32 image to a 1024-length bit vector (row-major).
    Any pixel > threshold -> 1; else 0.
    """
    g = wm_img.convert("L").resize((32, 32), Image.NEAREST)
    arr = np.array(g, dtype=np.uint8)
    bits = (arr > threshold).astype(np.uint8).ravel()
    if bits.size != 1024:
        raise ValueError("Watermark must resolve to exactly 32x32 = 1024 bits.")
    return bits

# ---------------------------
# DWT4: get/set LH4 (periodization)
# ---------------------------
def dwt4_get_LH4(Y: np.ndarray, wavelet='haar'):
    coeffs = pywt.wavedec2(Y, wavelet=wavelet, level=4, mode='periodization')
    # coeffs: [LL4, (LH4,HL4,HH4), (LH3,...), (LH2,...), (LH1,...)]
    LH4, HL4, HH4 = coeffs[1]
    return coeffs, LH4

def dwt4_set_LH4_and_recon(coeffs, LH4_new: np.ndarray, wavelet='haar') -> np.ndarray:
    # replace only LH4, keep HL4/HH4 as-is
    LH4, HL4, HH4 = coeffs[1]
    coeffs[1] = (LH4_new, HL4, HH4)
    Yw = pywt.waverec2(coeffs, wavelet=wavelet, mode='periodization')
    return Yw

# ---------------------------
# QIM on magnitude (sign-preserving)
# ---------------------------
def qim_embed_mag(c: float, b: int, Delta: float) -> float:
    s = -1.0 if c < 0 else 1.0
    m = abs(c)
    q = np.floor(m / Delta)
    m2 = (q + (0.25 if b == 0 else 0.75)) * Delta
    return s * m2

def qim_llr_mag(c: float, Delta: float) -> float:
    m = abs(c); x = m / Delta
    f = x - np.floor(x)
    return abs(f - 0.25) - abs(f - 0.75)  # <0 => bit 0 closer, >0 => bit 1 closer

# ---------------------------
# EMBED: LH4 first 64x64, 4 repeats
# ---------------------------
def embed_lh4_fixed64(
    host_img: Image.Image,
    wm32: Image.Image,
    alpha: float = 0.8,
    wavelet: str = 'haar'
) -> Image.Image:
    """
    Embed a 32x32 mono watermark (as bits) into the first 64x64 of LH4.
    Each bit is embedded 4 times (since 64*64=4096 = 1024*4).

    host must be >= 512x512 (any rectangular size OK).
    If LH4 is larger than 64x64 (e.g., 64x75), we only use [:64, :64].
    """
    # Convert host to YCbCr arrays
    Y, Cb, Cr = rgb_to_ycbcr_arrays(host_img)
    H, W = Y.shape

    # Sanity: host size
    if H < 512 or W < 512:
        raise ValueError(f"Host image too small ({H}x{W}); need at least 512x512.")

    # DWT level-4 and get LH4
    coeffs, LH4 = dwt4_get_LH4(Y, wavelet=wavelet)
    h4, w4 = LH4.shape
    if h4 < 64 or w4 < 64:
        raise ValueError(f"LH4 too small ({h4}x{w4}); need at least 64x64.")

    # Only use the first 64x64 region for embedding
    region = LH4[:64, :64].astype(np.float64).copy()

    # Δ from median magnitude in the region (robust scaling)
    med = np.median(np.abs(region))
    Delta = alpha * (med if med > 0 else 1.0)

    # Watermark -> bits and repeat 4x to fill 4096 coeffs exactly
    bits = wm32_to_bits(wm32)                    # length 1024
    R = 4
    if 64*64 != bits.size * R:
        raise RuntimeError("Internal size mismatch; 64*64 must equal 1024*4.")
    # Embed in raster order
    flat = region.ravel()
    for k in range(flat.size):                   # 0..4095
        b = int(bits[k % bits.size])             # repeat 4x
        flat[k] = qim_embed_mag(flat[k], b, Delta)
    region_e = flat.reshape(region.shape)

    # Put region back and reconstruct
    LH4_new = LH4.copy().astype(np.float64)
    LH4_new[:64, :64] = region_e
    Yw = dwt4_set_LH4_and_recon(coeffs, LH4_new, wavelet=wavelet)

    # Merge back to RGB
    out = merge_ycbcr_to_rgb(Yw, Cb, Cr)
    return out

def save_jpeg(input_path: str, out_path: str, quality: int = 75):
    Image.open(input_path).convert("RGB").save(out_path, quality=quality, subsampling=0, optimize=False)

# ---------------------------
# EXTRACT: mirror the same mapping
# ---------------------------
def extract_lh4_fixed64(
    wm_host_img: Image.Image,
    wavelet: str = 'haar',
    alpha: float = 0.8
) -> np.ndarray:
    """
    Blind extract 32x32 bits from LH4 first 64x64 (assumes 4 repeats).
    Returns an array shape (32, 32) with values {0,1}.
    """
    Y, _, _ = rgb_to_ycbcr_arrays(wm_host_img)
    coeffs, LH4 = dwt4_get_LH4(Y, wavelet=wavelet)
    h4, w4 = LH4.shape
    if h4 < 64 or w4 < 64:
        raise ValueError(f"LH4 too small ({h4}x{w4}); need at least 64x64.")

    region = LH4[:64, :64].astype(np.float64)
    # Recompute Δ from current region (blind)
    med = np.median(np.abs(region))
    Delta = alpha * (med if med > 0 else 1.0)

    flat = region.ravel()
    R = 4
    n_bits = 1024
    scores = np.zeros(n_bits, dtype=np.float64)

    # For bit i, its 4 positions are at indices:
    # i, i+1024, i+2048, i+3072  (since we wrote in raster)
    for i in range(n_bits):
        s = 0.0
        for r in range(R):
            k = i + r * n_bits
            s += qim_llr_mag(flat[k], Delta)
        scores[i] = s

    bits = (scores > 0).astype(np.uint8).reshape(32, 32)
    return bits

# ---------------------------
# Example usage (uncomment to run)
# ---------------------------
HOST_IMAGE_PATH = "images/cat.webp"
WATERMARK_IMAGE_PATH = "images/s.jpg"
OUTPUT_WATERMARK_PATH = "my-experiment/wm_recovered.png"
OUTPUT_WATERMARKED_PATH = "my-experiment/host_marked.png"

OUTPUT_JPEG_WATERMARK_PATH = "my-experiment/wm_recovered_compressed.jpg"
OUTPUT_JPEG_WATERMARKED_PATH = "my-experiment/host_marked_compressed.jpg"

host = Image.open(HOST_IMAGE_PATH)
wm   = Image.open(WATERMARK_IMAGE_PATH)  # mono logo/pattern
watermarked = embed_lh4_fixed64(host, wm, alpha=1.0, wavelet='haar')
watermarked.save(OUTPUT_WATERMARKED_PATH)

recovered_bits = extract_lh4_fixed64(watermarked, wavelet='haar', alpha=1.0)
# If you want a viewable 0/255 image:
rec_img = Image.fromarray((recovered_bits*255).astype(np.uint8))
rec_img.save(OUTPUT_WATERMARK_PATH)

psnr_value = psnr(cv2.imread(HOST_IMAGE_PATH), cv2.imread(OUTPUT_WATERMARKED_PATH))
print(f"PSNR: {psnr_value:.4f} dB")

# JPEG Attacks
jpeg_watermarked = Image.open(OUTPUT_JPEG_WATERMARKED_PATH)
save_jpeg(OUTPUT_WATERMARKED_PATH, OUTPUT_JPEG_WATERMARKED_PATH, quality=75)
recovered_bits = extract_lh4_fixed64(jpeg_watermarked, wavelet='haar', alpha=1.0)
# If you want a viewable 0/255 image:
rec_img = Image.fromarray((recovered_bits*255).astype(np.uint8))
rec_img.save(OUTPUT_JPEG_WATERMARK_PATH)

psnr_value = psnr(cv2.imread(HOST_IMAGE_PATH), cv2.imread(OUTPUT_JPEG_WATERMARKED_PATH))
print(f"PSNR: {psnr_value:.4f} dB")

PSNR: 44.3762 dB
PSNR: 36.4849 dB


# Grok

In [8]:
import numpy as np
from scipy.ndimage import gaussian_filter, maximum_filter, gaussian_laplace
from scipy.signal import wiener
from scipy.spatial import Delaunay

def harris_corner(image, sigma=1.0, k=0.05):
    Iy, Ix = np.gradient(image)
    Ix2 = gaussian_filter(Ix**2, sigma)
    Iy2 = gaussian_filter(Iy**2, sigma)
    Ixy = gaussian_filter(Ix*Iy, sigma)
    det = Ix2 * Iy2 - Ixy**2
    trace = Ix2 + Iy2
    r = det - k * trace**2
    return r

def get_feature_points(image, num_octaves=4, th=0.01):
    points = []
    current_image = image.astype(float).copy()
    for octave in range(num_octaves):
        sigma = 1.6 * (2 ** octave)
        r = harris_corner(current_image, sigma=sigma)
        local_max = maximum_filter(r, size=3) == r
        coords = np.nonzero(local_max & (r > th))
        for y, x in zip(*coords):
            lap = np.abs(gaussian_laplace(current_image, sigma))[y, x] * sigma**2
            lap_prev = np.abs(gaussian_laplace(current_image, sigma / 1.2))[y, x] * (sigma / 1.2)**2 if sigma > 1.6 else 0
            lap_next = np.abs(gaussian_laplace(current_image, sigma * 1.2))[y, x] * (sigma * 1.2)**2
            if lap > lap_prev and lap > lap_next:
                points.append((y * (2 ** octave), x * (2 ** octave), lap))
    # Sort by response (lap), take top 50
    points = sorted(points, key=lambda p: p[2], reverse=True)[:50]
    return np.array([p[:2] for p in points])

def barycentric_coords(tri, p):
    T = np.array([[tri[0,0] - tri[2,0], tri[1,0] - tri[2,0]],
                  [tri[0,1] - tri[2,1], tri[1,1] - tri[2,1]]])
    diff = p - tri[2]
    ab = np.linalg.solve(T, diff)
    g = 1 - ab[0] - ab[1]
    return np.array([ab[0], ab[1], g])

def compute_nvf(image):
    mean = gaussian_filter(image, 1.0)
    mean_sq = gaussian_filter(image**2, 1.0)
    variance = mean_sq - mean**2
    variance_max = variance.max() + 1e-9  # avoid division by zero
    nvf = variance / variance_max
    return nvf

def embed_watermark(image, watermark, alpha=0.2, beta=0.05, r=0.01):
    h, w = image.shape
    D = r * (w + h)
    points = get_feature_points(image)
    selected = []
    for p in points:
        if all(np.linalg.norm(p - q) > D for q in selected):
            selected.append(p)
    points = np.array(selected)
    if len(points) < 3:
        raise ValueError("Not enough feature points for watermarking.")
    delau = Delaunay(points)
    simplices = delau.simplices
    pts = delau.points
    watermarked = image.copy().astype(float)
    nvf = compute_nvf(image)
    for tri_idx in simplices:
        target_tri = pts[tri_idx]
        standard_tri = np.array([[0,0], [32,0], [0,32]], dtype=float)
        X = np.zeros((6,6))
        y = np.zeros(6)
        for i in range(3):
            x1, y1 = standard_tri[i]
            X[2*i, 0] = x1
            X[2*i, 1] = y1
            X[2*i, 2] = 1
            X[2*i+1, 3] = x1
            X[2*i+1, 4] = y1
            X[2*i+1, 5] = 1
            y[2*i] = target_tri[i,0]
            y[2*i+1] = target_tri[i,1]
        params = np.linalg.solve(X, y)
        a11, a12, b1, a21, a22, b2 = params
        A = np.array([[a11, a12, b1], [a21, a22, b2], [0,0,1]])
        A_inv = np.linalg.inv(A)
        min_x = int(np.min(target_tri[:,0]))
        max_x = int(np.max(target_tri[:,0])) + 1
        min_y = int(np.min(target_tri[:,1]))
        max_y = int(np.max(target_tri[:,1])) + 1
        for yy in range(max(min_y, 0), min(max_y, h)):
            for xx in range(max(min_x, 0), min(max_x, w)):
                p = np.array([xx, yy])
                bary = barycentric_coords(target_tri, p)
                if np.all(bary >= 0) and np.sum(bary) <= 1 + 1e-6:
                    p_hom = np.array([xx, yy, 1])
                    std_hom = A_inv @ p_hom
                    std_x = std_hom[0] / std_hom[2]
                    std_y = std_hom[1] / std_hom[2]
                    if 0 <= std_x < 32 and 0 <= std_y < 32:
                        wc = watermark[int(std_y), int(std_x)]
                        lam = alpha * (1 - nvf[yy, xx]) + beta * nvf[yy, xx]
                        watermarked[yy, xx] += lam * wc
    return np.clip(watermarked, 0, 255).astype(np.uint8) if image.dtype == np.uint8 else watermarked

def extract_watermark(image, watermark_size=32, alpha=0.2, beta=0.05, r=0.01):
    h, w = image.shape
    D = r * (w + h)
    points = get_feature_points(image.astype(float))
    selected = []
    for p in points:
        if all(np.linalg.norm(p - q) > D for q in selected):
            selected.append(p)
    points = np.array(selected)
    if len(points) < 3:
        return np.zeros((watermark_size, watermark_size))
    delau = Delaunay(points)
    simplices = delau.simplices
    pts = delau.points
    denoised = wiener(image.astype(float), mysize=(5,5))
    noise = image.astype(float) - denoised
    extracted = []
    for tri_idx in simplices:
        target_tri = pts[tri_idx]
        standard_tri = np.array([[0,0], [32,0], [0,32]], dtype=float)
        X = np.zeros((6,6))
        y = np.zeros(6)
        for i in range(3):
            x1, y1 = standard_tri[i]
            X[2*i, 0] = x1
            X[2*i, 1] = y1
            X[2*i, 2] = 1
            X[2*i+1, 3] = x1
            X[2*i+1, 4] = y1
            X[2*i+1, 5] = 1
            y[2*i] = target_tri[i,0]
            y[2*i+1] = target_tri[i,1]
        params = np.linalg.solve(X, y)
        a11, a12, b1, a21, a22, b2 = params
        A = np.array([[a11, a12, b1], [a21, a22, b2], [0,0,1]])
        A_inv = np.linalg.inv(A)
        wm_extract = np.zeros((watermark_size, watermark_size))
        for sy in range(watermark_size):
            for sx in range(watermark_size):
                p_std = np.array([sx, sy, 1])
                p_target = A @ p_std
                x_t = p_target[0] / p_target[2]
                y_t = p_target[1] / p_target[2]
                bary = barycentric_coords(target_tri, np.array([x_t, y_t]))
                if np.all(bary >= 0) and np.sum(bary) <= 1 + 1e-6:
                    xt = int(round(x_t))
                    yt = int(round(y_t))
                    if 0 <= xt < w and 0 <= yt < h:
                        wm_extract[sy, sx] = noise[yt, xt]
        extracted.append(wm_extract)
    if len(extracted) > 0:
        avg_wm = np.mean(extracted, axis=0)
    else:
        avg_wm = np.zeros((watermark_size, watermark_size))
    return avg_wm

# Example usage (commented out, as it requires an image)
# np.random.seed(42)
# watermark = 2 * (np.random.rand(32, 32) > 0.5) - 1  # Fixed monochrome watermark (+1 or -1)
# # Assume 'host_image' is a numpy array (grayscale)
# watermarked_image = embed_watermark(host_image, watermark)
# # Apply attacks (resize, rotation)
# from scipy.ndimage import zoom, rotate
# attacked = zoom(watermarked_image, 0.8)  # resize
# attacked = rotate(attacked, 30)  # rotation
# extracted_wm = extract_watermark(attacked)

In [10]:
import numpy as np
import cv2
from PIL import Image
from scipy.ndimage import zoom, rotate
from skimage import filters
import matplotlib.pyplot as plt

# Import your watermarking functions (assuming they're in a file called watermark.py)
# from watermark import embed_watermark, extract_watermark

def load_and_preprocess_images(host_path, watermark_path):
    """
    Load and preprocess host and watermark images
    """
    # Load host image and convert to grayscale
    host_image = cv2.imread(host_path, cv2.IMREAD_GRAYSCALE)
    if host_image is None:
        raise ValueError(f"Could not load host image from {host_path}")
    
    # Load watermark image
    watermark_img = cv2.imread(watermark_path, cv2.IMREAD_GRAYSCALE)
    if watermark_img is None:
        raise ValueError(f"Could not load watermark image from {watermark_path}")
    
    # Resize watermark to 32x32
    watermark_resized = cv2.resize(watermark_img, (32, 32), interpolation=cv2.INTER_AREA)
    
    # Convert watermark to binary using Otsu's thresholding
    threshold_value = filters.threshold_otsu(watermark_resized)
    watermark_binary = (watermark_resized > threshold_value).astype(np.float32)
    
    # Convert to +1/-1 format for watermarking
    watermark_binary = 2 * watermark_binary - 1
    
    return host_image, watermark_binary

def calculate_psnr(original, processed):
    """
    Calculate Peak Signal-to-Noise Ratio (PSNR) between two images
    """
    mse = np.mean((original.astype(np.float64) - processed.astype(np.float64)) ** 2)
    if mse == 0:
        return float('inf')
    
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_ber(original, extracted):
    """
    Calculate Bit Error Rate (BER) between original and extracted watermarks
    """
    # Convert to binary format for comparison
    original_bin = (original > 0).astype(int)
    extracted_bin = (extracted > 0).astype(int)
    
    # Calculate bit error rate
    errors = np.sum(original_bin != extracted_bin)
    total_bits = original_bin.size
    ber = errors / total_bits
    return ber

def apply_attacks(image, attack_type='rotation', param=5):
    """
    Apply various attacks to the watermarked image
    """
    if attack_type == 'rotation':
        # Rotate image by param degrees
        attacked = rotate(image, param, reshape=False, mode='constant', cval=0)
    elif attack_type == 'scaling':
        # Scale image by param factor
        attacked = zoom(image, param, order=1)
        # If scaled up, crop to original size
        if param > 1:
            h, w = image.shape
            start_h = (attacked.shape[0] - h) // 2
            start_w = (attacked.shape[1] - w) // 2
            attacked = attacked[start_h:start_h+h, start_w:start_w+w]
        # If scaled down, pad to original size
        elif param < 1:
            h, w = image.shape
            pad_h = (h - attacked.shape[0]) // 2
            pad_w = (w - attacked.shape[1]) // 2
            attacked = np.pad(attacked, ((pad_h, h-attacked.shape[0]-pad_h), 
                                       (pad_w, w-attacked.shape[1]-pad_w)), 
                            mode='constant', constant_values=0)
    elif attack_type == 'noise':
        # Add Gaussian noise with standard deviation param
        noise = np.random.normal(0, param, image.shape)
        attacked = image + noise
        attacked = np.clip(attacked, 0, 255)
    else:
        attacked = image
    
    return attacked.astype(image.dtype)

def visualize_results(host, watermark, watermarked, attacked, extracted):
    """
    Visualize the watermarking process and results
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Original host image
    axes[0, 0].imshow(host, cmap='gray')
    axes[0, 0].set_title('Host Image')
    axes[0, 0].axis('off')
    
    # Original watermark
    axes[0, 1].imshow(watermark, cmap='gray')
    axes[0, 1].set_title('Original Watermark')
    axes[0, 1].axis('off')
    
    # Watermarked image
    axes[0, 2].imshow(watermarked, cmap='gray')
    axes[0, 2].set_title('Watermarked Image')
    axes[0, 2].axis('off')
    
    # Attacked image
    axes[1, 0].imshow(attacked, cmap='gray')
    axes[1, 0].set_title('Attacked Image')
    axes[1, 0].axis('off')
    
    # Extracted watermark
    axes[1, 1].imshow(extracted, cmap='gray')
    axes[1, 1].set_title('Extracted Watermark')
    axes[1, 1].axis('off')
    
    # Difference between original and extracted
    diff = np.abs(watermark - extracted)
    axes[1, 2].imshow(diff, cmap='hot')
    axes[1, 2].set_title('Difference (Original vs Extracted)')
    axes[1, 2].axis('off')
    
    plt.tight_layout()
    plt.show()

def main():
    """
    Main function demonstrating the complete watermarking pipeline
    """
    # File paths (replace with your actual image paths)
    host_path = 'images/s.jpg'
    watermark_path = 'images/cat-wm.jpg'
    
    try:
        # Load and preprocess images
        print("Loading and preprocessing images...")
        host_image, watermark = load_and_preprocess_images(host_path, watermark_path)
        print(f"Host image shape: {host_image.shape}")
        print(f"Watermark shape: {watermark.shape}")
        
        # Embed watermark
        print("\nEmbedding watermark...")
        watermarked_image = embed_watermark(host_image, watermark, alpha=0.2, beta=0.05, r=0.01)
        
        # Calculate PSNR between host and watermarked image
        psnr_value = calculate_psnr(host_image, watermarked_image)
        print(f"PSNR (Host vs Watermarked): {psnr_value:.2f} dB")
        
        # Test without attacks (baseline)
        print("\nTesting extraction without attacks...")
        extracted_clean = extract_watermark(watermarked_image)
        ber_clean = calculate_ber(watermark, extracted_clean)
        print(f"BER (No attack): {ber_clean:.4f} ({ber_clean*100:.2f}%)")
        
        # Attack simulations
        attacks_to_test = [
            ('rotation', 5, 'Rotation 5°'),
            ('rotation', 10, 'Rotation 10°'),
            ('rotation', -5, 'Rotation -5°'),
            ('scaling', 0.8, 'Scaling 0.8x'),
            ('scaling', 1.2, 'Scaling 1.2x'),
            ('scaling', 0.9, 'Scaling 0.9x'),
            ('noise', 5, 'Gaussian Noise σ=5'),
            ('noise', 10, 'Gaussian Noise σ=10'),
        ]
        
        print("\nTesting robustness against attacks:")
        print("-" * 50)
        
        attack_results = []
        
        for attack_type, param, description in attacks_to_test:
            # Apply attack
            attacked_image = apply_attacks(watermarked_image, attack_type, param)
            
            # Extract watermark from attacked image
            extracted_watermark = extract_watermark(attacked_image)
            
            # Calculate BER
            ber = calculate_ber(watermark, extracted_watermark)
            
            # Calculate PSNR between watermarked and attacked image
            psnr_attack = calculate_psnr(watermarked_image, attacked_image)
            
            attack_results.append((description, ber, psnr_attack))
            print(f"{description:20s}: BER = {ber:.4f} ({ber*100:5.2f}%), PSNR = {psnr_attack:6.2f} dB")
        
        # Visualize one example (rotation attack)
        print("\nGenerating visualization for rotation attack...")
        attacked_example = apply_attacks(watermarked_image, 'rotation', 5)
        extracted_example = extract_watermark(attacked_example)
        
        visualize_results(host_image, watermark, watermarked_image, 
                         attacked_example, extracted_example)
        
        # Summary statistics
        print("\nSummary Statistics:")
        print("-" * 30)
        bers = [result[1] for result in attack_results]
        print(f"Average BER: {np.mean(bers):.4f} ({np.mean(bers)*100:.2f}%)")
        print(f"Min BER: {np.min(bers):.4f} ({np.min(bers)*100:.2f}%)")
        print(f"Max BER: {np.max(bers):.4f} ({np.max(bers)*100:.2f}%)")
        
        # Test with multiple watermarks for statistical analysis
        print("\nTesting with random watermarks for statistical analysis...")
        np.random.seed(42)
        n_tests = 10
        ber_stats = []
        
        for i in range(n_tests):
            # Generate random binary watermark
            random_watermark = 2 * (np.random.rand(32, 32) > 0.5) - 1
            
            # Embed and extract
            wm_img = embed_watermark(host_image, random_watermark)
            attacked_img = apply_attacks(wm_img, 'rotation', 5)  # 5-degree rotation
            extracted_wm = extract_watermark(attacked_img)
            
            ber = calculate_ber(random_watermark, extracted_wm)
            ber_stats.append(ber)
        
        print(f"Statistical analysis (n={n_tests}, rotation 5°):")
        print(f"Mean BER: {np.mean(ber_stats):.4f} ± {np.std(ber_stats):.4f}")
        print(f"Min BER: {np.min(ber_stats):.4f}")
        print(f"Max BER: {np.max(ber_stats):.4f}")
        
    except Exception as e:
        print(f"Error: {e}")
        print("\nNote: Make sure to:")
        print("1. Install required packages: pip install opencv-python pillow scikit-image")
        print("2. Replace 'host_image.jpg' and 'watermark.jpg' with actual file paths")
        print("3. Ensure the watermarking functions are imported correctly")

# Alternative: Generate synthetic images for testing
def generate_test_images():
    """
    Generate synthetic test images if real images are not available
    """
    print("Generating synthetic test images...")
    
    # Generate synthetic host image (256x256 with some texture)
    np.random.seed(42)
    host = np.random.rand(256, 256) * 100 + 100  # Base level + noise
    
    # Add some structure
    x, y = np.meshgrid(np.arange(256), np.arange(256))
    host += 50 * np.sin(x/20) * np.cos(y/30)  # Sinusoidal pattern
    host = np.clip(host, 0, 255).astype(np.uint8)
    
    # Generate synthetic watermark (text-like pattern)
    watermark = np.zeros((32, 32))
    # Create a simple pattern
    watermark[8:24, 8:24] = 1
    watermark[10:22, 10:22] = 0
    watermark[12:20, 12:20] = 1
    watermark = 2 * watermark - 1  # Convert to +1/-1
    
    return host, watermark


main()

host, watermark = generate_test_images()

# Test the watermarking pipeline
watermarked = embed_watermark(host, watermark)
psnr = calculate_psnr(host, watermarked)
print(f"PSNR: {psnr:.2f} dB")

# Test with rotation attack
attacked = apply_attacks(watermarked, 'rotation', 5)
extracted = extract_watermark(attacked)
ber = calculate_ber(watermark, extracted)
print(f"BER after 5° rotation: {ber:.4f} ({ber*100:.2f}%)")

Loading and preprocessing images...
Host image shape: (980, 980)
Watermark shape: (32, 32)

Embedding watermark...
PSNR (Host vs Watermarked): 62.81 dB

Testing extraction without attacks...


d:\ip-notebook\venv\lib\site-packages\scipy\signal\_signaltools.py:1673: RuntimeWarning: divide by zero encountered in divide
  res *= (1 - noise / lVar)
d:\ip-notebook\venv\lib\site-packages\scipy\signal\_signaltools.py:1673: RuntimeWarning: invalid value encountered in multiply
  res *= (1 - noise / lVar)


BER (No attack): 0.6641 (66.41%)

Testing robustness against attacks:
--------------------------------------------------
Rotation 5°         : BER = 0.6699 (66.99%), PSNR =   6.51 dB
Rotation 10°        : BER = 0.6592 (65.92%), PSNR =   5.69 dB


KeyboardInterrupt: 

In [6]:
import cv2
import numpy as np
import random

# --- Configuration Parameters ---
PATCH_SIZE = 32  # Size of the normalized patches (e.g., 32x32)
ALPHA = 10       # Watermark embedding strength
NUM_KEYPOINTS_TO_USE = 100 # Use the 100 most stable keypoints

def create_watermark(size, num_bits):
    """Generates a random binary sequence as the watermark."""
    return [random.randint(0, 1) for _ in range(num_bits)]

def get_normalized_patch(image, kp):
    """
    Extracts and normalizes a patch around a keypoint to be invariant
    to scale and rotation.
    """
    # Create a transformation matrix to deskew the patch
    # It rotates the patch to align with the keypoint's orientation
    # and scales it to a fixed size.
    m = cv2.getRotationMatrix2D(
        (kp.pt[0], kp.pt[1]),       # center
        kp.angle,                  # angle
        PATCH_SIZE / kp.size       # scale
    )
    # Adjust translation part of the matrix to center the patch
    m[0, 2] -= (kp.pt[0] - PATCH_SIZE / 2.0)
    m[1, 2] -= (kp.pt[1] - PATCH_SIZE / 2.0)

    # Apply the affine transformation to the grayscale image
    normalized_patch = cv2.warpAffine(
        image, m, (PATCH_SIZE, PATCH_SIZE), flags=cv2.INTER_LINEAR
    )
    return normalized_patch

def embed_watermark(image_path, watermark_bits):
    """
    Embeds the watermark into the image using SIFT features.
    """
    # 1. Read the image and convert to grayscale
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    watermarked_img = gray_img.copy().astype(np.float32)

    # 2. Detect SIFT keypoints
    sift = cv2.SIFT_create()
    keypoints, _ = sift.detectAndCompute(gray_img, None)
    
    # Sort keypoints by response (strongest first) and select the best ones
    keypoints = sorted(keypoints, key=lambda x: -x.response)[:NUM_KEYPOINTS_TO_USE]
    
    print(f"Embedding: Found {len(keypoints)} suitable keypoints.")
    if len(keypoints) < len(watermark_bits):
        raise ValueError("Not enough keypoints to embed the full watermark.")

    # 3. Embed one bit into each keypoint's patch
    num_bits = len(watermark_bits)
    for i, kp in enumerate(keypoints):
        bit_to_embed = watermark_bits[i % num_bits] # Embed redundantly

        # 4. Get the normalized patch for the keypoint
        patch = get_normalized_patch(watermarked_img, kp)
        
        # 5. Apply DCT and modify coefficients
        patch_dct = cv2.dct(patch.astype(np.float32))

        # Select two mid-frequency coefficients (example coordinates)
        c1_pos, c2_pos = (8, 9), (9, 8) 
        c1 = patch_dct[c1_pos]
        c2 = patch_dct[c2_pos]

        if bit_to_embed == 1:
            if c1 <= c2:
                c1 = c2 + ALPHA
        else: # bit_to_embed == 0
            if c2 <= c1:
                c2 = c1 + ALPHA
        
        patch_dct[c1_pos] = c1
        patch_dct[c2_pos] = c2

        # 6. Apply inverse DCT
        modified_patch = cv2.idct(patch_dct)

        # 7. Invert the normalization to place the patch back
        # This is a complex step often simplified by directly blending.
        # For simplicity, we'll skip the perfect inverse transform and blend.
        # A full implementation requires an inverse warp.
        # Here, we just acknowledge the concept.
        
    # In a full implementation, the modified patches would be carefully placed back.
    # For this conceptual code, we assume the modifications are small enough
    # that the `watermarked_img` (which was modified in-place by `get_normalized_patch` if not copied)
    # is the final product. A more robust method would be needed for a perfect reconstruction.
    
    # Convert back to a displayable format
    final_watermarked_img = cv2.convertScaleAbs(watermarked_img)
    return final_watermarked_img, keypoints


def extract_watermark(image, original_watermark_len):
    """
    Extracts the watermark from a (potentially attacked) image.
    """
    # 1. Convert to grayscale
    if len(image.shape) > 2:
        gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_img = image

    # 2. Detect SIFT keypoints
    sift = cv2.SIFT_create()
    keypoints, _ = sift.detectAndCompute(gray_img, None)
    keypoints = sorted(keypoints, key=lambda x: -x.response)[:NUM_KEYPOINTS_TO_USE]
    print(f"Extraction: Found {len(keypoints)} suitable keypoints.")

    # 3. Extract bits from each patch
    extracted_bits_collection = [[] for _ in range(original_watermark_len)]
    
    for kp in keypoints:
        patch = get_normalized_patch(gray_img, kp)
        patch_dct = cv2.dct(patch.astype(np.float32))

        # Use the same coefficient positions
        c1_pos, c2_pos = (8, 9), (9, 8)
        c1 = patch_dct[c1_pos]
        c2 = patch_dct[c2_pos]
        
        extracted_bit = 1 if c1 > c2 else 0
        
        # This part is simplified; a real system needs to match keypoints
        # to know *which* watermark bit it corresponds to. Without that,
        # we can use majority voting on the assumption of redundancy.
        # Here, we'll just collect them all.
        # For a robust system, we would match descriptors or use another locator.
        # This example assumes the order of strongest keypoints is somewhat preserved.
        
        # Store extracted bit for majority voting
        bit_index = len(extracted_bits_collection[0]) # Simplified mapping
        if bit_index < original_watermark_len:
             for i in range(original_watermark_len):
                 # This is a conceptual simplification of redundant bit recovery
                 extracted_bits_collection[i % original_watermark_len].append(extracted_bit)


    # 4. Reconstruct watermark using majority voting
    reconstructed_watermark = []
    for bits_for_one_pos in extracted_bits_collection:
        if not bits_for_one_pos:
            # Handle case where not enough keypoints were found
            reconstructed_watermark.append(0) 
            continue
        vote = sum(bits_for_one_pos) / len(bits_for_one_pos)
        reconstructed_watermark.append(1 if vote > 0.5 else 0)

    return reconstructed_watermark


if __name__ == "__main__":
    # --- Main Execution ---
    image_file = 'images/cat.webp'
    
    # Resize the image to 512x512
    img = cv2.imread(image_file)
    resized_img = cv2.resize(img, (512, 512))

    # 1. Generate a watermark
    watermark_length = 32
    original_watermark = create_watermark(1, watermark_length)
    print(f"Original Watermark:  {original_watermark}")

    # 2. Embed the watermark
    watermarked_image, _ = embed_watermark(image_file, original_watermark)
    cv2.imwrite("watermarked_lena.png", watermarked_image)
    print("Watermark embedded successfully into 'watermarked_lena.png'")

    # 3. Simulate an attack
    # Rotate by 30 degrees and scale by 80%
    rows, cols = watermarked_image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), 30, 0.8)
    attacked_image = cv2.warpAffine(watermarked_image, M, (cols, rows))
    cv2.imwrite("attacked_lena.png", attacked_image)
    print("Image attacked (rotated and scaled) and saved as 'attacked_lena.png'")

    # 4. Extract the watermark from the attacked image
    extracted_watermark = extract_watermark(attacked_image, watermark_length)
    print(f"Extracted Watermark: {extracted_watermark}")
    
    # 5. Compare results
    correct_bits = sum(1 for o, e in zip(original_watermark, extracted_watermark) if o == e)
    bit_error_rate = (watermark_length - correct_bits) / watermark_length
    print(f"\nComparison:")
    print(f"Correctly extracted bits: {correct_bits}/{watermark_length}")
    print(f"Bit Error Rate (BER): {bit_error_rate:.2%}")

Original Watermark:  [0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0]
Embedding: Found 100 suitable keypoints.
Watermark embedded successfully into 'watermarked_lena.png'
Image attacked (rotated and scaled) and saved as 'attacked_lena.png'
Extraction: Found 100 suitable keypoints.
Extracted Watermark: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Comparison:
Correctly extracted bits: 14/32
Bit Error Rate (BER): 56.25%


# Fourier–Mellin via log-polar DFT magnitude

In [25]:
import cv2
import numpy as np

# =========================
# Helpers
# =========================
def to_ycbcr(bgr):
    ycrcb = cv2.cvtColor(bgr, cv2.COLOR_BGR2YCrCb).astype(np.float32)
    Y, Cr, Cb = cv2.split(ycrcb)
    return Y, Cr, Cb

def from_ycbcr(Y, Cr, Cb, out_dtype=np.uint8):
    # Make all channels the same depth before merging
    Y  = Y.astype(np.float32)
    Cr = Cr.astype(np.float32)
    Cb = Cb.astype(np.float32)

    ycrcb = cv2.merge([
        np.clip(Y,  0, 255),
        np.clip(Cr, 0, 255),
        np.clip(Cb, 0, 255)
    ])
    bgr = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)
    return np.clip(bgr, 0, 255).astype(out_dtype)

def dft_mag_phase(img):
    # img: float32, single channel
    dft = cv2.dft(img, flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft, axes=(0, 1))
    re, im = dft_shift[...,0], dft_shift[...,1]
    mag = np.sqrt(re**2 + im**2) + 1e-8
    phase = np.arctan2(im, re)
    return mag, phase

def idft_from_mag_phase(mag, phase):
    re = mag * np.cos(phase)
    im = mag * np.sin(phase)
    dft_shift = np.stack([re, im], axis=-1)
    dft = np.fft.ifftshift(dft_shift, axes=(0, 1))
    img_back = cv2.idft(dft, flags=cv2.DFT_REAL_OUTPUT)
    return img_back

def logpolar(img, R=256, T=256):
    # img: float32
    h, w = img.shape[:2]
    center = (w/2.0, h/2.0)
    maxR = np.hypot(center[0], center[1])
    # OpenCV warpPolar with LOG converts radius to log scale automatically
    lp = cv2.warpPolar(img, (T, R), center, maxR,
                       flags=cv2.WARP_POLAR_LOG + cv2.WARP_FILL_OUTLIERS)
    return lp, center, maxR

def inv_logpolar(lp, center, maxR, out_hw):
    h, w = out_hw
    cart = cv2.warpPolar(lp, (w, h), center, maxR,
                         flags=cv2.WARP_POLAR_LOG + cv2.WARP_INVERSE_MAP + cv2.WARP_FILL_OUTLIERS)
    return cart

def circ_xcorr_fft(a, b):
    # circular cross-correlation using FFT, returns shift index where a is shifted to best match b
    A = np.fft.fft(a)
    B = np.fft.fft(b)
    cc = np.fft.ifft(A * np.conj(B)).real
    return int(np.argmax(cc))

def binarize_32x32(wm):
    # Expect 32x32 grayscale/mono; output {0,1}
    wm = cv2.resize(wm, (32, 32), interpolation=cv2.INTER_AREA)
    if wm.dtype != np.uint8:
        wm = np.clip(wm, 0, 255).astype(np.uint8)
    _, bw = cv2.threshold(wm, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return bw.astype(np.uint8)

# =========================
# Scheme constants (public, no external input needed)
# =========================
R, T = 256, 256              # log-polar grid (rows=radius bins, cols=angle bins)
RING_RADIUS = 28             # ring pilot row index (safe from DC; tune 20~40)
RADIAL_ANGLE = 3             # radial pilot column index (very thin wedge)
PILOT_ALPHA = 0.015          # pilot strength in log-magnitude domain
BAND_R_MIN, BAND_R_MAX = 48, 160  # payload band (rows) ~ mid frequencies
DELTA = 0.8                  # QIM step in log-mag units
MOVE_RATIO = 0.7             # move tile mean partially towards quantizer center

# Fixed PN sequences (public, compiled-in)
rng = np.random.default_rng(12345)
PN_ANG = rng.choice([-1.0, 1.0], size=T).astype(np.float32)        # ring pilot pattern across angles
PN_RAD = rng.choice([-1.0, 1.0], size=R).astype(np.float32)        # radial pilot pattern across radius

# =========================
# Core: Embed
# =========================
def embed_watermark(bgr_host, wm32_mono):
    """
    bgr_host: uint8 HxWx3, any rectangular size. Not resized.
    wm32_mono: uint8 32x32 (0..255 or 0/1). We'll binarize with Otsu.
    returns: uint8 watermarked BGR, same size as host
    """
    H, W = bgr_host.shape[:2]
    Y, Cr, Cb = to_ycbcr(bgr_host.astype(np.uint8))
    Yf = Y.astype(np.float32)

    # DFT → log-magnitude
    mag, phase = dft_mag_phase(Yf)
    logmag = np.log(mag + 1e-8)

    # To log-polar
    lp, center, maxR = logpolar(logmag, R=R, T=T)

    # 1) Embed pilots
    lp[RING_RADIUS, :] += PILOT_ALPHA * PN_ANG
    col = (RADIAL_ANGLE % T)
    lp[:, col] += PILOT_ALPHA * PN_RAD

    # 2) Payload mapping (32x32 tiles inside [BAND_R_MIN, BAND_R_MAX] × full angle)
    bits = binarize_32x32(wm32_mono).reshape(32, 32)
    rows_band = BAND_R_MAX - BAND_R_MIN
    assert rows_band >= 32, "Increase R or adjust band to fit 32 radial tiles."

    # compute tile sizes
    tile_r = rows_band // 32
    tile_t = T // 32

    for i in range(32):
        r0 = BAND_R_MIN + i * tile_r
        r1 = r0 + tile_r
        if i == 31: r1 = BAND_R_MAX  # last tile absorbs remainder

        for j in range(32):
            t0 = j * tile_t
            t1 = t0 + tile_t if j < 31 else T

            tile = lp[r0:r1, t0:t1]
            if tile.size == 0:
                continue

            # QIM on tile mean (log-magnitude)
            b = int(bits[i, j])
            m = float(tile.mean())

            # dithered QIM centers for bit 0/1 (±Δ/4 offset)
            target = DELTA * np.round((m) / DELTA) + (DELTA/4 if b==1 else -DELTA/4)

            # move mean partially toward target to control distortion
            delta = (target - m) * MOVE_RATIO
            lp[r0:r1, t0:t1] = tile + delta

    # Inverse log-polar → exp → iDFT with original phase
    mod_logmag = inv_logpolar(lp, center, maxR, out_hw=(H, W))
    mag_new = np.exp(mod_logmag)  # undo log

    Y_new = idft_from_mag_phase(mag_new, phase)
    Y_new = np.clip(Y_new, 0, 255).astype(np.uint8)
    return from_ycbcr(Y_new, Cr, Cb)

# =========================
# Core: Blind Extract
# =========================
def extract_watermark(bgr_watermarked):
    """
    Blind extraction. Input: only the watermarked BGR image.
    Output: recovered 32x32 watermark (uint8 0/255)
    """
    H, W = bgr_watermarked.shape[:2]
    Y, _, _ = to_ycbcr(bgr_watermarked.astype(np.uint8))
    Yf = Y.astype(np.float32)

    # DFT → log-magnitude → log-polar
    mag, _ = dft_mag_phase(Yf)
    logmag = np.log(mag + 1e-8)
    lp, center, maxR = logpolar(logmag, R=R, T=T)

    # === Resynchronization via pilots ===
    # 1) Rotation (horizontal circular shift) from ring pilot row
    ring_row = np.clip(RING_RADIUS, 0, R-1)
    obs_ang = lp[ring_row, :].astype(np.float32)
    shift_theta = circ_xcorr_fft(obs_ang, PN_ANG)
    lp = np.roll(lp, -shift_theta, axis=1)  # undo rotation

    # 2) Scale (vertical shift) from radial pilot column
    col = (RADIAL_ANGLE % T)
    obs_rad = lp[:, col].astype(np.float32)
    shift_scale = circ_xcorr_fft(obs_rad, PN_RAD)
    lp = np.roll(lp, -shift_scale, axis=0)  # undo scale

    # === Read payload (QIM decision) ===
    rows_band = BAND_R_MAX - BAND_R_MIN
    tile_r = rows_band // 32
    tile_t = T // 32

    bits = np.zeros((32, 32), dtype=np.uint8)
    for i in range(32):
        r0 = BAND_R_MIN + i * tile_r
        r1 = r0 + tile_r
        if i == 31: r1 = BAND_R_MAX

        for j in range(32):
            t0 = j * tile_t
            t1 = t0 + tile_t if j < 31 else T
            tile = lp[r0:r1, t0:t1]
            if tile.size == 0:
                bits[i, j] = 0
                continue

            m = float(tile.mean())
            # QIM decision: even vs odd bin (via offset sign)
            # Compute distance to two nearest centers of 0 and 1 hypotheses
            c0 = DELTA * np.round(m / DELTA) - DELTA/4
            c1 = DELTA * np.round(m / DELTA) + DELTA/4
            bits[i, j] = 1 if abs(m - c1) < abs(m - c0) else 0

    # Return as 0/255 image
    return (bits * 255).astype(np.uint8)

# =========================
# (Optional) PSNR
# =========================
def psnr(a, b):
    a = a.astype(np.float64); b = b.astype(np.float64)
    mse = np.mean((a - b)**2)
    return 99.0 if mse < 1e-12 else 10.0 * np.log10((255.0**2) / mse)


In [26]:
# read your files
host = cv2.imread("images/cat.webp")            # any WxH color image
wm   = cv2.imread("images/cat-wm.jpg", 0)    # 32x32 mono (or any mono; it will be binarized)

wm_img = embed_watermark(host, wm)
cv2.imwrite("watermarked.png", wm_img)

# BLIND extraction (needs only the watermarked image)
recovered = extract_watermark(cv2.imread("watermarked.png"))
cv2.imwrite("recovered_32x32.png", recovered)

# Fix the PSNR calculation - compare host with watermarked image
psnr_val = psnr(host, wm_img)
print(f"PSNR: {psnr_val:.2f} dB")  # expect ~40dB+ for mild embedding

# Display results
print(f"Original watermark shape: {wm.shape}")
print(f"Recovered watermark shape: {recovered.shape}")

# Show the images (optional)
cv2.imshow("Original Host", host)
cv2.imshow("Watermarked", wm_img)
cv2.imshow("Original Watermark", wm)
cv2.imshow("Recovered Watermark", recovered)
cv2.waitKey(0)
cv2.destroyAllWindows()

PSNR: 4.13 dB
Original watermark shape: (400, 400)
Recovered watermark shape: (32, 32)
